In [ ]:
from data import *
from utils import *
import pandas as pd
import pygwalker as pyg

import neurokit2 as nk

train_data = np.load('/workspace/signal/PVC-NET_/dataset//mit-bih-arrhythmia-database-1.0.0_train_20230216.npy',allow_pickle=True) # B x (C) x Signal
valid_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-arrhythmia-database-1.0.0_valid_20230216.npy',allow_pickle=True) # B x (C) x Signal
test_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-arrhythmia-database-1.0.0_test_20230216.npy',allow_pickle=True) # B x (C) x Signal

AMC_data  = np.load('/workspace/signal/PVC-NET_/dataset/AMC_PeakLabel_3rd_125Hz.npy',allow_pickle=True) # 497 samples
CPSC2020_data  = np.load('/workspace/signal/PVC-NET_/dataset/CPSC2020_testSeg_ver2.npy',allow_pickle=True)
CU_data  = np.load('/workspace/signal/PVC-NET_/dataset/cu-ventricular-tachyarrhythmia-database-1.0.0_testSeg.npy',allow_pickle=True)
ESC_data  = np.load('/workspace/signal/PVC-NET_/dataset/european-st-t-database-1.0.0_testSeg.npy',allow_pickle=True)
FANTASIA_data = np.load('/workspace/signal/PVC-NET_/dataset/fantasia-database-1.0.0_testSeg.npy', allow_pickle=True) # B x (C) x Signal
INCART_data  = np.load('/workspace/signal/PVC-NET_/dataset/INCART_testSeg.npy',allow_pickle=True)
NS_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-noise-stress-test-database-1.0.0_testSeg.npy',allow_pickle=True)
STDB_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-st-change-database-1.0.0_testSeg.npy',allow_pickle=True)
SVDB_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-supraventricular-arrhythmia-database-1.0.0_testSeg.npy',allow_pickle=True)

In [ ]:
df = pd.read_csv(, parse_dates=['date'])
gwalker = pyg.walk(df)

In [ ]:
trainvalid_data = np.concatenate((train_data, valid_data))
np.save('mit-bih-arrhythmia-database-1.0.0_trainvalid_2023.npy',)

In [ ]:
len(train_data), len(valid_data), len(test_data), len(trainvalid_data)

In [ ]:
train_data = np.concatenate((train_data,STDB_data))
len(train_data)

In [ ]:
# data = FANTASIA_data
data = CU_data

print(len(data))
for i in range(len(data)):
    if len(data[i]['symbol_PVC'])!=0:
        print(i,data[i]['symbol_PVC'])

In [ ]:
!pip install monai==1.0.0

In [ ]:
import monai
monai.__version__

In [1]:
import torch
import models
from models.unet import *
from models.densenet import *
from models.resnet import *
from models.efficientnet import *
from models.ffc import *
import monai

In [ ]:
net = densenet121(spatial_dims =1, in_channels=1, out_channels=2, module='SCM12')
net

In [ ]:
# net = resnet34(n_input_channels=1, spatial_dims=1, module='acm')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='cbam')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='deeprft')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='ffc')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='mha')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='nlnn')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='se')
net = resnet34(n_input_channels=1, spatial_dims=1, module='SCM12')

In [2]:
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='acm')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='cbam')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='deeprft')
# net = EfficientNetBNFeatures(in_channels=1, model_name = 'efficientnet-b0', spatial_dims=1, module='ffc')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='nlnn')
net = EfficientNetBNFeatures(in_channels=1, model_name = 'efficientnet-b0', spatial_dims=1, module='SESCM12')


AssertionError: 

In [ ]:
x = torch.rand(1,1,1280)
yhat = net(x)

In [ ]:
for yh in yhat:
    print(yh.shape)

In [4]:
net = UNet(spatial_dims =1, modelName= 'basic', in_channels=1, out_channels=2)

UNET encoder is basic
[64, 128, 256, 512, 512]
U-NET encModule is NONE
U-NET mtl is NONE
U-NET decModule is NONE
U-NET supervision is NONE
U-NET segheadModule is NONE


In [5]:
net

UNet(
  (conv_0): TwoConv(
    (conv_0): Convolution(
      (conv): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (adn): ADN(
        (N): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (D): Dropout(p=0.1, inplace=False)
        (A): GELU()
      )
    )
    (conv_1): Convolution(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (adn): ADN(
        (N): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (D): Dropout(p=0.1, inplace=False)
        (A): GELU()
      )
    )
  )
  (down_1): Down(
    (max_pooling): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (convs): TwoConv(
      (conv_0): Convolution(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (adn): ADN(
          (N): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (D): Dropout(p=0